In [4]:
import os
import pandas as pd
import openpyxl
import datetime

def extract_student_codes(base_path):
    course_student_codes = {}
    
    courses = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    
    for course in courses:
        exams_path = os.path.join(base_path, course, "Exams")
        student_codes = set()
        
        if os.path.exists(exams_path):
            submissions = os.listdir(exams_path)
            
            for submission in submissions:
                parts = submission.split('_')
                if len(parts) > 0:
                    student_code = parts[0]
                    student_codes.add(student_code)
        
        course_student_codes[course] = student_codes
    
    return course_student_codes

base_path = r"C:\Users\zl516\Box\1_WPDOCS\1 - Exams & Grades Team\Exams\Exam4 Exam Files\202410 Take Home Final Exams\202410 Take Home Final Exams- Faculty"
student_data = extract_student_codes(base_path)

flat_data = [(course, code) for course, codes in student_data.items() for code in codes]
box_submission = pd.DataFrame(flat_data, columns=['Course Box Folder Name', 'Exam Code'])

box_submission['Submission Key'] = box_submission.iloc[:, 0] + '_' + box_submission.iloc[:, 1]
unique_key = pd.read_excel('exam_unique_key.xlsx')
exam_database = pd.read_csv('take-home_examinfo.csv')[['Course & Instructor','Course #','Exam #']]

exam_database = pd.merge(exam_database, unique_key[['Course #', 'Box Folder Name']], on='Course #', how='left')
exam_database['Exam #'] = exam_database['Exam #'].astype(str)
exam_database['Submission Key'] = exam_database.iloc[:, 3] + '_' + exam_database.iloc[:, 2]

missing_submissions = exam_database[~exam_database['Submission Key'].isin(box_submission['Submission Key'])]

not_in_database = box_submission.merge(exam_database[['Submission Key']], on='Submission Key', how='left', indicator=True)
not_in_database = not_in_database[not_in_database['_merge'] == 'left_only'].drop(columns=['_merge'])

common_entries = pd.merge(exam_database, box_submission[['Submission Key']], on='Submission Key', how='inner')

unique_confirmed_submissions = common_entries.drop_duplicates(subset='Submission Key')

overview_data = {
    'Description': ['Total Exams Registered', 'Submissions Received', 'Confirmed Submissions', 'Confirmed Submissions Unique', 'Missing Submissions', 'Incorrect Submissions'],
    'Count': [len(exam_database), len(box_submission), len(common_entries), len(unique_confirmed_submissions), len(missing_submissions), len(not_in_database)]
}
overview_df = pd.DataFrame(overview_data)

with pd.ExcelWriter('Exam_Submission_Details.xlsx') as writer:
    overview_df.to_excel(writer, sheet_name='Overview', index=False)
    exam_database.to_excel(writer, sheet_name='All Exams Registered', index=False)
    box_submission.to_excel(writer, sheet_name='Submissions Received', index=False)
    common_entries.to_excel(writer, sheet_name='Confirmed Submissions', index=False)
    unique_confirmed_submissions.to_excel(writer, sheet_name='Confirmed Submissions Unique', index=False)
    missing_submissions.to_excel(writer, sheet_name='Missing Submissions', index=False)
    not_in_database.to_excel(writer, sheet_name='Incorrect Submissions', index=False)

print("Data saved to Exam_Submission_Details.xlsx")

Data saved to Exam_Submission_Details.xlsx
